#Driver's Drowsiness Detection System

In [ ]:
import cv2
import dlib
from scipy.spatial import distance as dist
import time
import threading
import winsound

In [ ]:
# Function to calculate eye aspect ratio (EAR)
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

In [ ]:
# Function to calculate mouth aspect ratio (MAR)
def mouth_aspect_ratio(mouth):
    A = dist.euclidean(mouth[14], mouth[18])
    C = dist.euclidean(mouth[12], mouth[16])
    mar = A / C
    return mar

In [ ]:
# Thresholds for eye aspect ratio (EAR) and mouth aspect ratio (MAR)
EAR_THRESHOLD = 0.25
MAR_THRESHOLD = 0.5
CLOSED_EYES_DURATION_THRESHOLD = 2  # in seconds

# Initialize the dlib face detector and facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [ ]:
# Initialize the video capture object
cap = cv2.VideoCapture(0)

# Initialize variables for drowsiness detection
start_time_closed = None
alert_playing = False

In [ ]:
# Function to play alert sound asynchronously
def play_alert_sound():
    winsound.PlaySound('final_alarm.wav', winsound.SND_FILENAME)

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    faces = detector(gray)

    for face in faces:
        # Get facial landmarks
        shape = predictor(gray, face)
        landmarks = [(shape.part(i).x, shape.part(i).y) for i in range(68)]

        # Extract eye and mouth landmarks
        left_eye = landmarks[36:42]
        right_eye = landmarks[42:48]
        mouth = landmarks[48:68]

        # Calculate eye aspect ratio (EAR) for each eye
        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)
        avg_ear = (left_ear + right_ear) / 2

        # Calculate mouth aspect ratio (MAR)
        mar = mouth_aspect_ratio(mouth)

        # Check if eyes are closed based on EAR
        if avg_ear < EAR_THRESHOLD:
            if start_time_closed is None:
                start_time_closed = time.time()
            elif time.time() - start_time_closed > CLOSED_EYES_DURATION_THRESHOLD:
                drowsy_eyes = True
        else:
            start_time_closed = None
            drowsy_eyes = False

        # Check if yawning is detected based on MAR
        if mar > MAR_THRESHOLD:
            drowsy_yawn = True
        else:
            drowsy_yawn = False

        # If eyes are closed for more than 2 seconds or both eye closure and yawning are detected, play alert sound asynchronously
        if drowsy_eyes or (drowsy_eyes and drowsy_yawn):
            if not alert_playing:
                alert_thread = threading.Thread(target=play_alert_sound)
                alert_thread.start()
                alert_playing = True
        else:
            alert_playing = False

        # Draw rectangle around the face
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Draw rectangle around the eyes
        for (x, y) in left_eye:
            cv2.circle(frame, (x, y), 2, (0, 0, 255), -1)
        for (x, y) in right_eye:
            cv2.circle(frame, (x, y), 2, (0, 0, 255), -1)

        # Draw rectangle around the mouth
        for (x, y) in mouth:
            cv2.circle(frame, (x, y), 2, (255, 0, 0), -1)

    # Display the frame
    cv2.imshow('Frame', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture object and close all windows
cap.release()
cv2.destroyAllWindows()